# pandas 进阶修炼 ｜早起Python
<br>

**本习题由公众号【早起Python & 可视化图鉴】 原创，转载及其他形式合作请与我们联系（微信号`sshs321`)，未经授权严禁搬运及二次创作，侵权必究！**



本习题基于 `pandas` 版本 `1.1.3`，所有内容应当在 `Jupyter Notebook` 中执行以获得最佳效果。


不同版本之间写法可能会有少许不同，如若碰到此情况，你应该学会如何自行检索解决。

## 9 - 更多未提及的操作



<br>


**<font color = '#5172F0'><font size=3.5>必读👇👇👇</font>**
    
    
    

## 初始化

<br>

该 `Notebook` 版本为**习题+答案版**


请执行下方单元格以初始化读取本节习题的答案

执行完毕后，你可以使用 `ans(1)` 来查看第 1 题的参考解答，其余习题类似。

注意：所有答案并非固定（唯一），我提供的答案仅供参考（并非推荐答案或者最优解）

如果发现任何错误或者你有更好的解答，可以微信搜索公众号「早起Python」提交以获得奖励！


In [72]:
import sys
# sys.path.append('../document/')
# from init import initialize, ans
# res = initialize("../document/answer-9.txt")

## 9-1 `map` 与 `applymap`

<br>

`pandas` 中的 `map` 和 `applymap` 可以对指定列（map）或整个数据框（applymap）工作

完成替换、格式化、计算等操作，是 `Pandas` 数据分析中十分重要的工具。

为了方便理解，首先执行下方代码创建并查看数据

In [81]:
import pandas as pd
import numpy as np

df1 = pd.DataFrame({'A': ['A0', 'A1', np.nan, 'A3'],
                    'B': ['B0',np.nan,'B3',np.nan],
                    'C': ['C0','C1','C2',np.nan],
                    'D': np.random.randn(4),
                    'E': np.random.randn(4),
                   'F': np.random.randn(4)},
                   index=[0, 1, 2, 3])

### 1 - map｜基本使用

将 `df1` 第一列中的 `A0` 替换为 `cat`，`A3` 替换为 `rabbit`，其余为设置为`NaN`（缺失值）

In [82]:
df1['A'] = df1['A'].map({'A0':'cat','A3':'rabbit'})

### 2 - map｜匿名函数

在上一题的结果上，将 df1 第 1 列中的字符末尾追加「今天关注了早起Python」

In [83]:
df1['A'] = df1['A'].map(lambda x:f'{x} 今天关注了早起Python')

### 3 - map｜跳过缺失值

上一题中，nan（缺失值）也被同步追加了字符串

现在重新对第二列执行同样的操作，并跳过缺失值

In [84]:
df1['B'] = df1['B'].map(lambda x:f'{x} 今天关注了早起Python', na_action='ignore')

### 4 - map｜自定义函数

除了 lambda ，map还可以接受自定义函数，现在对第三列，使用自定义函数完成上一题的任务

In [85]:
def mapfun(x):
    
    return str(x) + "今天关注了早起Python"

In [86]:
df1['C'] = df1['C'].map(mapfun, na_action='ignore')

### 5 - applymap｜lambda

`applymap`可以对整个 `dataframe` 工作，现在将 df1 的最后三列保留两位小数

In [91]:
df1[['F','D','E']] = df1[['F','D','E']].applymap(lambda x:"%.2f" % x)

## 9-2  `stack` 与 `unstack` 

### 6 - stack｜数据堆叠

<br>

stack字面意思是数据堆叠，但是理解起来就是将数据由宽变长

怎样做到？

通过**将部分列名拿下来当作索引**来实现，例如下图所示

本来应是`2列4行`，但通过 `stack` 可以将列A拿下来当作索引，从而变成`1列8行`

![](https://pandas.pydata.org/docs/_images/reshaping_stack.png)

为了复现上面的例子，首先需要执行下方代码来生成数据

In [92]:
tuples = list(
    zip(
        *[
            ["bar", "bar", "baz", "baz", "foo", "foo", "qux", "qux"],
            ["one", "two", "one", "two", "one", "two", "one", "two"],
        ]
    )
)
index = pd.MultiIndex.from_tuples(tuples, names=["first", "second"])
df = pd.DataFrame(np.arange(1,17).reshape([8,2]), index=index, columns=["A", "B"])
df2 = df[:4]

现在，按上图所示，对 df2 进行堆叠

In [95]:
stacked = df2.stack()

### 7 - unstack｜逆堆叠

对上一题的结果进行还原，即逆堆叠，过程如下图所示

![](https://pandas.pydata.org/docs/_images/reshaping_unstack.png)

In [99]:
stacked.unstack()

A  B
first second      
bar   one     1  2
      two     3  4
baz   one     5  6
      two     7  8

### 8 - unstack｜层级

在使用 unstack 进行逆堆叠时，可以指定层级，例如指定按照 second 进行，也就是如下图所示

![](https://pandas.pydata.org/docs/_images/reshaping_unstack_1.png)

In [100]:
stacked.unstack('second')

second   one  two
first            
bar   A    1    3
      B    2    4
baz   A    5    7
      B    6    8

## 9-3 `isin` 筛选

在 `pandas` 中有没有类似 `SQL` 中 `IN` 和 `NOTIN` 的筛选方法？

`isin`就可以实现，通过 isin 可以快速筛选出包含某个值的结果

为了方便练习，首先需要执行下面的代码生成示例数据，并应简单查看一下

In [103]:
df3 = pd.DataFrame({'country': ['China','US', 'UK', 'Germany', 'Japan'],
             'rank':[1,2,3,4,5]})

### 9 - isin｜根据列表筛选

筛选出 `country` 包含 `'China','UK'` 的行

In [105]:
df3[df3.country.isin(['China','UK'])]

,country,rank
0,China,1
2,UK,3


### 10 - isin｜逆筛选

对上一题的结果取逆

In [ ]:
df3[~df3.country.isin(['China','UK'])]

,country,rank
1,US,2
3,Germany,4
4,Japan,5


## 9-4  `select_dtypes` 筛选

<br>

`select_dtypes`  可以筛选制定数据类型的列

为了方便练习，首先需要执行下面的代码生成示例数据，并应简单查看一下

In [ ]:
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.select_dtypes.html

In [107]:
df4 = pd.DataFrame({'a': [1, 2] * 3,
                   'b': [True, False] * 3,
                   'c': [1.0, 2.0] * 3})

### 11 - select_dtypes｜单类型

筛选 df4 数据类型为整数的列

In [108]:
df4.select_dtypes(include=['int64'])

,a
0,1
1,2
2,1
3,2
4,1
5,2


### 12 - select_dtypes｜多类型

筛选 df4 数据类型为和浮点数的列

In [109]:
df4.select_dtypes(include=['int','float64'])

,a,c
0,1,1.0
1,2,2.0
2,1,1.0
3,2,2.0
4,1,1.0
5,2,2.0


### 13 - select_dtypes｜排除

筛选 df4 数据类型为布尔值的列

In [110]:
df4.select_dtypes(exclude=['int','float64'])

,b
0,True
1,False
2,True
3,False
4,True
5,False


##  9-5 `explode` 数据展开

<br>

有时我们的数据中会包含列表，此时便可使用  `explode` 进行展开，将一个list拆成多行

为了方便练习，首先需要执行下面的代码生成示例数据，并应简单查看一下

In [111]:
df5 = pd.DataFrame({'A': [[0, 1, 2], 'foo', [], [3, 4]],
                   'B': 1,
                   'C': [['a', 'b', 'c'], np.nan, [], ['d', 'e']]})

In [112]:
df5

,A,B,C
0,"[0, 1, 2]",1,"[a, b, c]"
1,foo,1,NaN
2,[],1,[]
3,"[3, 4]",1,"[d, e]"


### 14 - explode｜单列

将 df5 第 A 列进行展开

In [114]:
df5.explode('A')

,A,B,C
0,0,1,"[a, b, c]"
0,1,1,"[a, b, c]"
0,2,1,"[a, b, c]"
1,foo,1,NaN
2,NaN,1,[]
3,3,1,"[d, e]"
3,4,1,"[d, e]"


### 15 - explode｜多列

将 df5 第 A、C 列进行展开

In [116]:
# df5.explode(list('AC')) # pandas版本 >= 1.3 才可以完成

## 9-6 `nunique` 统计

<br>

`nunique` 可以统计指定轴上不唯一的元素数量

[👉对应官方文档](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.nunique.html)

为了方便练习，首先需要执行下面的代码生成示例数据，并应简单查看一下

In [117]:
df6 = pd.DataFrame({'A': [4, 5, 6], 'B': [4, 1, 1]})

### 16 - nunique｜按列

In [118]:
df6.nunique()

A    3
B    2
dtype: int64

### 17 - nunique｜按行

In [119]:
df6.nunique(axis=1)

0    1
1    2
2    2
dtype: int64

## 9-7  `cumsum` 计算

cumsum 可以对数据按照指定方式进行累加

[👉官方文档](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.cumsum.html)

为了方便练习，首先需要执行下面的代码生成示例数据，并应简单查看一下

In [125]:
df7 = pd.DataFrame(np.arange(1,37).reshape([9,4]), columns=["A", "B","C","D"])
df7['item'] = ['Apple','Xiaomi','Huawei'] * 3

### 18 - cumsum｜按列

将 df7 按列进行累加

In [126]:
df7[list('ABCD')].cumsum()

,A,B,C,D
0,1,2,3,4
1,6,8,10,12
2,15,18,21,24
3,28,32,36,40
4,45,50,55,60
5,66,72,78,84
6,91,98,105,112
7,120,128,136,144
8,153,162,171,180


### 19 - cumsum｜按行

将 df7 按行进行累加

In [ ]:
df[list('ABCD')].cumsum(axis = 1)

### 20 - cumsum｜按组

将 df7 按照 `item` 按不同组对第 A 列进行累加

In [ ]:
df7 = df.sort_values(['item']).reset_index(drop=True)
df7['cusum']=df.groupby('item')['A'].cumsum(axis=0)

## 9-8 `append`｜添加

在很多教程，包括 [pandas 官方文档](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html#appending-rows-to-a-dataframe)中，都将 append 结合 merge、concat、join 一起讲解

但是对我来说，虽然append得到的结果也类似合并，可它常常出现的地方就是它的字面意思 -> 添加（追加）

所以我将在这里介绍 append

下面是几个 append 的常用操作，为了方便练习，首先需要执行下面的代码生成示例数据，并应简单查看一下



In [127]:
df8 = pd.DataFrame(
    {
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    },
    index=[0, 1, 2, 3],
)

s2 = pd.Series(["X0", "X1", "X2", "X3"], index=["A", "B", "C", "D"])
s3 = pd.DataFrame({"A": ['s1'],"B": ['s2'],"C": ['s3'],"D": ['s4']})
dicts = [{"A": 1, "B": 2, "C": 3, "X": 4}, {"A": 5, "B": 6, "C": 7, "Y": 8}]

### 21 - append｜末尾追加

将 s2 添加至 df8 的末尾

![](https://pandas.pydata.org/pandas-docs/stable/_images/merging_append_series_as_row.png)

In [129]:
result = df8.append(s2, ignore_index=True)

### 22 - append｜指定位置追加

将 s3 添加至 df8 的第三行

In [130]:
df9 = df8.iloc[:2, :]
df10 = df8.iloc[2:, :]

pd.concat([df9, s3, df10])

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
0,s1,s2,s3,s4
2,A2,B2,C2,D2
3,A3,B3,C3,D3


### 23 - append｜添加字典

将下面的字典 dicts 插入添加至 df8，并保留索引，如下图所示

![](https://pandas.pydata.org/pandas-docs/stable/_images/merging_append_dits.png)

In [131]:
result = df8.append(dicts, ignore_index=True, sort=False)

## 9-9 `compare` 比较

<br>

`compare` 用于比较两个数据框之间的差异

[👉官方文档](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.compare.html)


为了方便练习，首先需要执行下面的代码生成示例数据，并应简单查看一下

In [132]:
df9 = pd.DataFrame(
    {
        "col1": ["a", "a", "b", "b", "a"],
        "col2": [1.0, 2.0, 3.0, np.nan, 5.0],
        "col3": [1.0, 2.0, 3.0, 4.0, 5.0]
    },
    columns=["col1", "col2", "col3"],
)


df10 = df9.copy()
df10.loc[0, 'col1'] = 'c'
df10.loc[2, 'col3'] = 4.0


### 24 - compare｜常规

输出 df9 和 df10 的差异

In [134]:
df9.compare(df10)

col1       col3      
  self other self other
0    a     c  NaN   NaN
2  NaN   NaN  3.0   4.0

### 25 - compare｜保留数据框

在上一题的要求下，保留原数据框

In [135]:
df9.compare(df10, keep_shape=True)

col1       col2       col3      
  self other self other self other
0    a     c  NaN   NaN  NaN   NaN
1  NaN   NaN  NaN   NaN  NaN   NaN
2  NaN   NaN  NaN   NaN  3.0   4.0
3  NaN   NaN  NaN   NaN  NaN   NaN
4  NaN   NaN  NaN   NaN  NaN   NaN

### 26 - compare｜保留值

在上一题的基础上，再保留原始相同的值

In [136]:
df9.compare(df10, keep_shape=True, keep_equal=True)

col1       col2       col3      
  self other self other self other
0    a     c  1.0   1.0  1.0   1.0
1    a     a  2.0   2.0  2.0   2.0
2    b     b  3.0   3.0  3.0   4.0
3    b     b  NaN   NaN  4.0   4.0
4    a     a  5.0   5.0  5.0   5.0